In [1]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import numpy as np
import os
import time
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

In [2]:
os.listdir('../토지특성정보/')

['AL_11_D195_20191023.csv',
 'AL_11_D195_20200925.csv',
 'AL_11_D195_20210521.csv',
 'AL_11_D195_20210819.csv',
 'AL_11_D195_20211104.csv',
 '경기도']

In [3]:
def create_land_specs_df():
    basedir = '../토지특성정보/'
    filenames = [f for f in os.listdir(basedir) if (f.endswith('.csv'))&('AL_' in f)]
    
    dfs_list = []
    for file in tqdm(filenames):
        df = pd.read_csv(basedir + file, encoding='euc-kr')
        
        if '기준연도' in df.columns:
            df = df.rename(columns={'기준연도':'기준년도'})
        
        dfs_list.append(df)
    
    concat_df = pd.concat(dfs_list).drop_duplicates().reset_index(drop=True)
    
    return concat_df

In [4]:
land_specs_df = create_land_specs_df()
print(land_specs_df.shape)
land_specs_df.head()

  0%|                                                                                            | 0/5 [00:00<?, ?it/s]c:\users\max6296\ml\lib\site-packages\IPython\core\interactiveshell.py:3338: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [01:53<00:00, 22.73s/it]


(9529966, 26)


,고유번호,법정동코드,법정동명,대장구분코드,대장구분명,지번,토지일련번호,기준년도,기준월,지목코드,지목명,토지면적,용도지역코드1,용도지역명1,용도지역코드2,용도지역명2,토지이용상황코드,토지이동상황,지형높이코드,지형높이,지형형상코드,지형형상,도로접면코드,도로접면,공시지가,데이터기준일자
0,1111010100100010000,1111010100,서울특별시 종로구 청운동,1.0,일반,1,1,2013,1,8,대,15622.1,13,제1종일반주거지역,0,지정되지않음,120,연립,2,평지,4,사다리형,6,소로한면,2840000,2017-10-17
1,1111010100100010000,1111010100,서울특별시 종로구 청운동,1.0,일반,1,1,2014,1,8,대,15622.1,13,제1종일반주거지역,0,지정되지않음,140,아파트,2,평지,4,사다리형,6,소로한면,2933000,2017-10-17
2,1111010100100010000,1111010100,서울특별시 종로구 청운동,1.0,일반,1,1,2015,1,8,대,15622.1,13,제1종일반주거지역,0,지정되지않음,140,아파트,2,평지,4,사다리형,6,소로한면,3053000,2017-10-17
3,1111010100100010000,1111010100,서울특별시 종로구 청운동,1.0,일반,1,1,2016,1,8,대,15622.1,13,제1종일반주거지역,0,지정되지않음,140,아파트,2,평지,4,사다리형,6,소로한면,3234000,2017-10-17
4,1111010100100010000,1111010100,서울특별시 종로구 청운동,1.0,일반,1,1,2017,1,8,대,15622.1,13,제1종일반주거지역,0,지정되지않음,140,아파트,2,평지,4,사다리형,6,소로한면,3488000,2018-05-23


In [5]:
cols_to_drop = [col for col in land_specs_df.columns if '코드' in col]
cols_to_drop

['법정동코드',
 '대장구분코드',
 '지목코드',
 '용도지역코드1',
 '용도지역코드2',
 '토지이용상황코드',
 '지형높이코드',
 '지형형상코드',
 '도로접면코드']

In [6]:
land_specs_df = land_specs_df.drop(columns=cols_to_drop + ['고유번호']).drop_duplicates().reset_index(drop=True)
print(land_specs_df.shape)
land_specs_df.head()

(8549302, 16)


,법정동명,대장구분명,지번,토지일련번호,기준년도,기준월,지목명,토지면적,용도지역명1,용도지역명2,토지이동상황,지형높이,지형형상,도로접면,공시지가,데이터기준일자
0,서울특별시 종로구 청운동,일반,1,1,2013,1,대,15622.1,제1종일반주거지역,지정되지않음,연립,평지,사다리형,소로한면,2840000,2017-10-17
1,서울특별시 종로구 청운동,일반,1,1,2014,1,대,15622.1,제1종일반주거지역,지정되지않음,아파트,평지,사다리형,소로한면,2933000,2017-10-17
2,서울특별시 종로구 청운동,일반,1,1,2015,1,대,15622.1,제1종일반주거지역,지정되지않음,아파트,평지,사다리형,소로한면,3053000,2017-10-17
3,서울특별시 종로구 청운동,일반,1,1,2016,1,대,15622.1,제1종일반주거지역,지정되지않음,아파트,평지,사다리형,소로한면,3234000,2017-10-17
4,서울특별시 종로구 청운동,일반,1,1,2017,1,대,15622.1,제1종일반주거지역,지정되지않음,아파트,평지,사다리형,소로한면,3488000,2018-05-23


In [7]:
land_specs_df.to_csv('./Prepped Data/land_specs_baseline_20211116.csv', index=False)